In [4]:
# import needed modules
import time, random

# define our functions
def inc(x):
    time.sleep(random.random())
    return x + 1

def dec(x):
    time.sleep(random.random())
    return x - 1

def add(x, y):
    time.sleep(random.random())
    return x + y

In [3]:
%%time

# a sequential example with no parallelization
results = []
for x in range(20):
    result = inc(x)
    result = dec(result)
    results.append(result)

print(results)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
CPU times: user 0 ns, sys: 3.51 ms, total: 3.51 ms
Wall time: 22.5 s


In [1]:
# import dask for parallel work
from dask.distributed import Client, LocalCluster

In [2]:
# Set up our cluster with default workers and threads
cluster = LocalCluster(processes=False)
cluster

LocalCluster(45bdbfe7, 'inproc://10.244.1.57/30673/1', workers=1, threads=8, memory=31.34 GiB)

In [5]:
%%time
# Set up a client for work
client = cluster.get_client()

results = []
for x in range(20):
    result = client.submit(inc, x)
    result = client.submit(dec, result)
    results.append(result)

results = client.gather(results)
print(results)
client.close()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
CPU times: user 332 ms, sys: 12.6 ms, total: 344 ms
Wall time: 3.66 s


In [6]:
# When we are done we can close our dask cluster
cluster.close()

In [7]:
# Set up a new cluster with default 8 workers and 1 thread per worker
cluster = LocalCluster(n_workers=8, processes=False, threads_per_worker=1)
cluster

LocalCluster(cdcaf30a, 'inproc://10.244.1.57/30673/10', workers=8, threads=8, memory=31.34 GiB)

In [8]:
%%time
# Set up a client for work
client = cluster.get_client()

results = []
for x in range(20):
    result = client.submit(inc, x)
    result = client.submit(dec, result)
    results.append(result)

results = client.gather(results)
print(results)
client.close()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
CPU times: user 549 ms, sys: 77.6 ms, total: 627 ms
Wall time: 3.11 s


In [9]:
# When we are done we can close our dask cluster
cluster.close()

In [13]:
import coiled # use a Coiled cluster

In [17]:
# Set up cluster in the cloud where we can grab more workers
cluster = coiled.Cluster(n_workers=20) # run on a cluster in the cloud
client = cluster.get_client()

Output()

myst-parser 0.18.1 has requirement mdit-py-plugins~=0.3.1, but you have mdit-py-plugins 0.4.0.

Package - myst-parser, Pip check had the following issues that need resolving: 
myst-parser 0.18.1 has requirement mdit-py-plugins~=0.3.1, but you have mdit-py-plugins 0.4.0.

╭──────────────────────────────── Package Info ────────────────────────────────╮
│                            ╷                                                 │
│   Package                  │ Note                                            │
│ ╶──────────────────────────┼───────────────────────────────────────────────╴ │
│   coiled_local_jovyan      │ Source wheel built from /home/jovyan            │
│                            ╵                                                 │
╰──────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────── Not Synced with Cluster ───────────────────────────╮
│               ╷                                                  ╷           │
│   Package     │ Error                                            │ Risk      │
│ ╶─────────────┼──────────────────────────────────────────────────┼─────────╴ │
│   myst-parser │ Pip check had the following issues that need     │ Warning   │
│               │ resolving:                                       │           │
│               │ myst-parser 0.18.1 has requirement               │           │
│               │ mdit-py-plugins~=0.3.1, but you have             │           │
│               │ mdit-py-plugins 0.4.0.                           │           │
│               ╵                                                  ╵           │
╰──────────────────────────────────────────────────────────────────────────────╯

Output()

In [18]:
%%time
results = []
for x in range(20): # scale 100x
    result = client.submit(inc, x)
    result = client.submit(dec, result)
    results.append(result)

results = client.gather(results)
print(results)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
CPU times: user 136 ms, sys: 11.4 ms, total: 148 ms
Wall time: 2.33 s


Set up on coiled with free GCP (or AWS or Azure)
* go to coiled.io
* sign-in
* Open a terminal and run
```
conda deactivate
conda create -n coiled -c conda-forge coiled google-cloud-sdk
conda install -c conda-forge google-cloud-sdk -n notebook
conda activate coiled
coiled setup gcp
```

In [ ]:
```
(coiled) jovyan@jupyter-eeholmes:~$ coiled setup aws
╭────────────────────────────────────────────────────────────────────────────────────────╮
│ Introduction                                                                           │
│                                                                                        │
│ This uses your AWS credentials to set up Coiled.                                       │
│                                                                                        │
│ This will do the following ...                                                         │
│ 1. Create limited IAM roles and grant them to Coiled                                   │
│ 2. Check and expand your AWS quota if needed                                           │
│ 3. Create initial resources to deploy clusters                                         │
│                                                                                        │
│ This will not ...                                                                      │
│ 1. Create resources that cost money                                                    │
│ 2. Grant Coiled access to your data                                                    │
╰────────────────────────────────────────────────────────────────────────────────────────╯
Missing: You don't have local AWS credentials.
That's ok, you can run setup from AWS CloudShell.

Run setup from AWS CloudShell with the following steps:

1. Go to https://console.aws.amazon.com/cloudshell
2. Sign in to your AWS account
   (if you usually switch role or profile, you should do this)
3. Run the following command in CloudShell:

  pip3 install coiled && \ 
  coiled login \ 
    --token b2bfb562be4a457da2a74efa \
    --token 42713203-d012864e09b94db \
    --token b82f66c7c273c6913213a9fa9 && \
  coiled setup aws --region us-east-1
```

In [16]:
coiled setup aws --quotas

SyntaxError: invalid syntax (3831637974.py, line 1)

In [19]:
627/148

4.236486486486487

In [20]:
20/8

2.5

In [21]:
cluster

Output()

╭─────────────────────────────── Coiled Cluster ───────────────────────────────╮
│           ]8;id=311123;https://cloud.coiled.io/clusters/445452?account=eeholmes\https://cloud.coiled.io/clusters/445452?account=eeholmes]8;;\           │
╰──────────────────────────────────────────────────────────────────────────────╯
╭────────────── Overview ──────────────╮╭─────────── Configuration ────────────╮
│                                      ││                                      │
│ Name: eeholmes-ef8daa0e              ││ Region: us-east-1                    │
│                                      ││                                      │
│ Scheduler Status: started            ││ Scheduler: m6i.xlarge                │
│                                      ││                                      │
│ Dashboard:                           ││ Workers:   m6i.xlarge (15), Unknown  │
│ ]8;id=89740;https://cluster-qajgq.dask.host?token=rKD-dj80QhjCSklb\https://cluster-qajgq.dask.host?toke]8;;\ ││ (5)                                  │
│ ]8;id=89740;https://cluster-qajgq.dask.host?token=rKD-dj80QhjCSklb\n=rKD-dj80QhjCSklb]8;;\                   ││                                      │
│                                      ││ Workers Requested: 20                │
╰──────────────────────────────────────╯╰──────────────────────────────────────╯
╭──────────────── Dask Worker States (2024/04/16 19:09:24 UTC) ────────────────╮
│     Provisioning            ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0       │
│     Booting Instance        ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0       │
│     Downloading environment ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0       │
│     Ready                   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━ 15      │
│     Stopping                ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0       │
│     Stopped                 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0       │
│     Error                   ━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5       │
╰──────────────────────────────────────────────────────────────────────────────╯